### JOLTS Report

Updated October 6, 2020 with new JOLTS series IDs:
https://www.bls.gov/jlt/jlt_series_changes.htm

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

ind = {'000000': '\\textbf{Total nonfarm}',
       '110099': 'Mining \& logging',
       '230000': 'Construction',
       '320000': 'Durable goods manufacturing',
       '340000': 'Nondurable goods manufacturing',
       '510000': 'Information',
       '510099': 'Financial activities',
       '540099': 'Professional \& business services',
       '440000': 'Retail trade',
       '480099': 'Transportation, warehousing, \& utilities',
       '610000': 'Educational services',
       '620000': 'Health care \& social services',
       '720000': 'Accommodation \& food services',
       '910000': 'Federal government',
       '920000': 'State \& local government'}

In [2]:
series = {'JTS000000000000000JOL': 'Openings', 
          'JTS000000000000000HIL': 'Hires', 
          'JTS000000000000000QUL': 'Quits',
          'JTS000000000000000TSL': 'Separations',
          'JTS000000000000000LDL': 'Layoffs',
          'LNS13000000': 'Unemp'}

series = jolts_codes(series, 'QUR', ind)
    
df1 = bls_api(series, (2000, 2021), bls_key)

df1.to_csv(data_dir/'jolts_master.csv', index_label='date')

series2 = {}
series = jolts_codes(series2, 'JOR', ind)
    
df2 = bls_api(series2, (2000, 2021), bls_key)

df2.to_csv(data_dir/'jolts_master2.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED
Post Request Status: REQUEST_SUCCEEDED


In [3]:
df = (pd.read_csv(data_dir / 'jolts_master.csv', parse_dates=['date'])
        .set_index('date'))

levels_srs = ['Openings', 'Hires', 'Quits', 'Separations', 'Unemp', 'Layoffs']
rates_srs = ['JTS720000000000000QUR', 'JTS000000000000000QUR']

levels_data = df[levels_srs].dropna() / 1000
levels_data.to_csv(data_dir / 'jolts.csv', index_label='date')

unjo = levels_data['Openings'] / levels_data['Unemp']
unjo.name = 'unjo'
unjo.to_csv(data_dir / 'unjo.csv', index_label='date')

ltdata = levels_data.iloc[-1]

ltdate = dtxt(ltdata.name)['mon1']
write_txt(text_dir / 'jolts_ltdate.txt', ltdate)
ltopen = ltdata['Openings']
pryropen = levels_data['Openings'].iloc[-13]
lthire = ltdata['Hires']
ltquit = ltdata['Quits']
ltsep = ltdata['Separations']
ltun = ltdata['Unemp']
ltlay = ltdata['Layoffs']
pryrlay = levels_data['Layoffs'].iloc[-13]
pryrdate = dtxt(levels_data.index[-13])['mon1']

text = (f'In {ltdate}, there were {ltopen:.1f} million total nonfarm job '+
        'openings (see {\color{blue!80!black}\\textbf{---}}) '
        f'and {lthire:.1f} million hires completed '+
        '(see {\color{cyan}\\textbf{---}}). In the same month '+
        f'there were {ltsep:.1f} million nonfarm separations, of '+
        f'which {ltlay:.1f} million were layoffs '+
        '(see {\color{orange!70!yellow}\\textbf{---}}) '+
        f'and {ltquit:.1f} million were voluntary '+
        '(see {\color{red!85!violet}\\textbf{---}}). '+
        f'One year prior, in {pryrdate}, there were {pryrlay:.1f} '+
        f'million layoffs, and {pryropen:.1f} million job openings.')
write_txt(text_dir / 'jolts2.txt', text)
print(text)

In August 2021, there were 10.4 million total nonfarm job openings (see {\color{blue!80!black}\textbf{---}}) and 6.3 million hires completed (see {\color{cyan}\textbf{---}}). In the same month there were 6.0 million nonfarm separations, of which 1.3 million were layoffs (see {\color{orange!70!yellow}\textbf{---}}) and 4.3 million were voluntary (see {\color{red!85!violet}\textbf{---}}). One year prior, in August 2020, there were 1.6 million layoffs, and 6.5 million job openings.


In [4]:
ratio = ltdata['Openings'] / ltdata['Unemp']
ratiop = levels_data['Openings'].iloc[-13] / levels_data['Unemp'].iloc[-13]
p3date = dtxt(levels_data.index[-37])['mon1']
ratio3 = levels_data['Openings'].iloc[-37] / levels_data['Unemp'].iloc[-37]

text = ('For additional context, '+
        f'there are {ltun:.1f} million unemployed persons in {ltdate}. The ratio '+
        f'of job openings to unemployed persons was {ratio:.1f} in {ltdate} '+
        '(see {\color{magenta!80!violet!90}\\textbf{---}}), '+
        f'compared to {ratiop:.1f} in the same month one year prior, and '+
        f'{ratio3:.1f} in {p3date}.')
write_txt(text_dir / 'unjo.txt', text)
node = end_node(unjo, 'magenta!70!violet', percent=False, date=True)
write_txt(text_dir / 'unjo_node.txt', node)
print(text)

For additional context, there are 8.4 million unemployed persons in August 2021. The ratio of job openings to unemployed persons was 1.2 in August 2021 (see {\color{magenta!80!violet!90}\textbf{---}}), compared to 0.5 in the same month one year prior, and 1.2 in August 2018.


In [5]:
short_names = {'JTS000000000000000QUR': 'TOT_QU', 'JTS720000000000000QUR': 'AFS_QU'}
rates_data = df[rates_srs].dropna().rename(short_names, axis=1)

rates_data.to_csv(data_dir / 'quits.csv', index_label='date')

node = end_node(rates_data['AFS_QU'], 'red!50!purple', date=True)
write_txt(text_dir / 'quits_afs_node.txt', node)

node = end_node(rates_data['TOT_QU'], 'violet!90!black')
write_txt(text_dir / 'quits_tot_node.txt', node)

ltdata = rates_data.iloc[-1]

afs_max = rates_data['AFS_QU'].max()
afs_idxmax = dtxt(rates_data['AFS_QU'].idxmax())['mon1']

text = (f'In {ltdate}, the total quits rate in all industries was {ltdata.TOT_QU} percent'+
        ' (see {\color{violet!90!black}\\textbf{---}}). The '+
        f'accommodations and food services quits rate was {ltdata.AFS_QU} '+
        'percent (see {\color{red!50!purple}\\textbf{---}}); the series '+
        f'high for the industry group was {afs_max} percent in {afs_idxmax}.')

write_txt(text_dir / 'quits_afs.txt', text)

print(text)

In August 2021, the total quits rate in all industries was 2.9 percent (see {\color{violet!90!black}\textbf{---}}). The accommodations and food services quits rate was 6.8 percent (see {\color{red!50!purple}\textbf{---}}); the series high for the industry group was 6.8 percent in August 2021.


In [6]:
df = (pd.read_csv(data_dir / 'jolts_master.csv', parse_dates=['date'])
        .set_index('date'))

max_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().max()
min_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().min()
max_qur2 = df.loc['2018':,df.columns.str.endswith('QUR')].dropna().max()
min_qur2 = df.loc['2018':,df.columns.str.endswith('QUR')].dropna().min()
lt_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().iloc[-1]

qur = pd.DataFrame({'max': max_qur, 'min': min_qur, 'latest': lt_qur, 
                    'maxst': max_qur2, 'minst': min_qur2})

quits = {}
quits = jolts_codes(quits, 'QUR', ind, value='name')
    
final = pd.DataFrame()
final['outer1'] = qur['min']
final['range1'] = qur['minst'] - qur['min']
final['rangest'] = qur['maxst'] - qur['minst']
final['range2'] = qur['max'] - qur['maxst']
final['outer2'] = qur['max'].max() - qur['max']
final['latest'] = qur['latest']
final.index = final.index.map(quits)
final = final.sort_values('latest', ascending=False)
final['y'] = [i * -1 + 0.08 for i in list(range(0, len(qur)))]
final.index.name = 'name'
final.to_csv(data_dir / 'quits_ind.csv', sep=';')

final

,outer1,range1,rangest,range2,outer2,latest,y
name,,,,,,,
Accommodation \& food services,2.6,0.8,3.4,0.0,0.0,6.8,0.08
Retail trade,1.7,0.5,2.5,0.0,2.1,4.7,-0.92
Professional \& business services,1.5,0.6,1.6,0.0,3.1,3.4,-1.92
Nondurable goods manufacturing,0.8,0.6,1.7,0.0,3.7,3.0,-2.92
\textbf{Total nonfarm},1.2,0.4,1.3,0.0,3.9,2.9,-3.92
Construction,0.7,0.6,1.4,0.6,3.5,2.7,-4.92
Health care \& social services,1.2,0.4,1.1,0.0,4.1,2.7,-5.92
"Transportation, warehousing, \& utilities",0.8,0.8,1.7,0.0,3.5,2.3,-6.92
Durable goods manufacturing,0.5,0.1,1.5,0.0,4.7,2.1,-7.92


In [7]:
df = (pd.read_csv(data_dir / 'jolts_master2.csv', parse_dates=['date'])
        .set_index('date'))

max_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().max()
min_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().min()
max_jor2 = df.loc['2018':,df.columns.str.endswith('JOR')].dropna().max()
min_jor2 = df.loc['2018':,df.columns.str.endswith('JOR')].dropna().min()
lt_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().iloc[-1]

jor = pd.DataFrame({'max': max_jor, 'min': min_jor, 'latest': lt_jor, 
                    'maxst': max_jor2, 'minst': min_jor2})

jos = {}
jos = jolts_codes(jos, 'JOR', ind, value='name')
    
final = pd.DataFrame()
final['outer1'] = jor['min']
final['range1'] = jor['minst'] - jor['min']
final['rangest'] = jor['maxst'] - jor['minst']
final['range2'] = jor['max'] - jor['maxst']
final['outer2'] = jor['max'].max() - jor['max']
final['latest'] = jor['latest']
final.index = final.index.map(jos)
final = final.sort_values('latest', ascending=False)
final['y'] = [i * -1 + 0.08 for i in list(range(0, len(jor)))]
final.index.name = 'name'
final.to_csv(data_dir / 'openings_ind.csv', sep=';')

final

,outer1,range1,rangest,range2,outer2,latest,y
name,,,,,,,
Accommodation \& food services,1.5,2.1,7.7,0.0,0.0,10.2,0.08
Professional \& business services,2.3,1.9,3.9,0.0,3.2,7.9,-0.92
"Transportation, warehousing, \& utilities",0.8,3.1,3.9,0.0,3.5,7.8,-1.92
Nondurable goods manufacturing,0.9,1.8,5.0,0.0,3.6,7.7,-2.92
Retail trade,1.3,2.0,3.9,0.0,4.1,7.2,-3.92
Health care \& social services,2.4,2.0,3.7,0.0,3.2,7.1,-4.92
\textbf{Total nonfarm},1.7,1.7,3.6,0.0,4.3,6.6,-5.92
Durable goods manufacturing,0.6,1.5,4.2,0.0,5.0,5.8,-6.92
Mining \& logging,0.5,1.0,4.5,0.0,5.3,5.3,-7.92
